<a href="https://colab.research.google.com/github/Francisogbankwa/projects.ibuilt/blob/main/Stock_Market_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import tensorflow as tf
from tkinter import *

from testing import predict_stock


def next_day_price(stock_data):
    # Split the data into features and labels
    X = stock_data[:, :-1]
    y = stock_data[:, -1]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Train a linear regression model
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    # Predict the next day's stock price
    next_day_features = X_test[-1, :]
    next_day_price = lr.predict([next_day_features])

    return next_day_price[0]


class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self, X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:, :, i])
            self.scalers.append(Scaler)
        return X

    def transform(self, X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:, :, i])
        return X


def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length):
        X.append(data.iloc[i: i + feature_length, :].values)
        Y.append(data["Close"].values[i + feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X, Y


def PredictStockPrice(model, stock_symbol, start_date):
    # Load the stock data
    stock_data = yf.download(stock_symbol, start=start_date, interval='1d')

    # Sort the data points based on indexes
    stock_data.sort_index(inplace=True)

    # Remove any duplicate index
    stock_data = stock_data.loc[~stock_data.index.duplicated(keep='first')]

    # Filter only required data
    stock_data = stock_data[['Close', 'Volume']]

    # Confirm the testing set length
    test_length = stock_data[(stock_data.index >= '2020-09-01')].shape[0]

    # Create the features and targets
    X, Y = CreateFeatures_and_Targets(stock_data, 32)

    # Create the scalers
    Feature_Scaler = MultiDimensionScaler()
    X = Feature_Scaler.fit_transform(X)

    Target_Scaler = MinMaxScaler()
    Y = Target_Scaler.fit_transform(Y.reshape(-1, 1))

    # Split into training and testing sets
    Xtrain, Xtest, Ytrain, Ytest = X[:-test_length], X[-test_length:], Y[:-test_length], Y[-test_length:]

    # Make predictions using the trained model
    Predictions = model.predict(Xtest)
    Predictions = Target_Scaler.inverse_transform(Predictions)

    # Plot the results
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=stock_data.index[-len(Predictions):], y=Predictions[:, 0], name='Predicted Price'))
    fig.add_trace(go.Scatter(x=stock_data.index[-len(Ytest):], y=Ytest[:, 0], name='Actual Price'))
    fig.update_layout(title=f"{stock_symbol} Predicted Stock Price", xaxis_title="Date", yaxis_title="Price")
    fig.show()

    # Return the predicted prices
    return Predictions

def predict_stock():
    stock_symbol = stock_entry.get()
    start_date = '2010-01-01'
    model = tf.keras.models.load_model('LSTM_Model')
    predictions = PredictStockPrice(model, stock_symbol, start_date)
    print(predictions)

# create a tkinter window
root = Tk()
root.geometry("400x200")

# create a label and entry for the stock symbol
Label(root, text="Enter stock symbol: ").grid(row=0, column=0)
stock_entry = Entry(root)
stock_entry.grid(row=0, column=1)

# create a button to predict the stock
predict_button = Button(root, text="Predict Stock", command=predict_stock)
predict_button.grid(row=1, column=1)

root.mainloop()
